In [ ]:
import os
import sys
src_dir = os.path.join(os.getcwd())
abs_path = os.path.abspath(os.path.join(src_dir, os.pardir, 'src'))
sys.path.append(abs_path)

from utils import GLOBAL

In [ ]:
import pandas as pd

In [ ]:
#Projections for current year
proj_df = pd.read_csv(GLOBAL.PROJECTIONS_2023)

#ADP for current year and scoring format
adp_df = pd.read_csv(GLOBAL.ADP_2023_HALF_PPR)

#Either Standard, Half PPR, or PPR. Match with the ADP data above
FANTASY_SCORING_FORMAT = "Half PPR"

#Scoring Weights based on scoring format above
scoring_weights = {}
if FANTASY_SCORING_FORMAT == "Standard":
    scoring_weights = GLOBAL.STANDARD_SCORING
elif FANTASY_SCORING_FORMAT == "Half PPR":
    scoring_weights = GLOBAL.HALF_PPR_SCORING
elif FANTASY_SCORING_FORMAT == "PPR":
    scoring_weights = GLOBAL.PPR_SCORING

#Sanitize data
proj_df = proj_df.replace(',','', regex=True)
proj_df = proj_df.replace({'Pos': '0'}, {'Pos': 'QB'})

proj_df[['Receptions', 'ReceivingYds', 'ReceivingTD', 'RushingYds', 'RushingTD', 'FumblesLost', 'PassingYds', 'PassingTD', 'Interceptions']] = proj_df[['Receptions', 'ReceivingYds', 'ReceivingTD', 'RushingYds', 'RushingTD', 'FumblesLost', 'PassingYds', 'PassingTD', 'Interceptions']].astype(float)

proj_df['FantasyPoints'] = (
    (proj_df['Receptions'] * scoring_weights['Receptions']) +
    (proj_df['ReceivingYds'] * scoring_weights['ReceivingYds']) +
    (proj_df['ReceivingTD'] * scoring_weights['ReceivingTD']) +
    (proj_df['RushingYds'] * scoring_weights['RushingYds']) +
    (proj_df['RushingTD'] * scoring_weights['RushingTD'] )+
    (proj_df['FumblesLost'] * scoring_weights['FL']) +
    (proj_df['PassingYds'] * scoring_weights['PassingYds']) +
    (proj_df['PassingTD'] * scoring_weights['PassingTD']) +
    (proj_df['Interceptions'] * scoring_weights['Int'])
    )

proj_df[FANTASY_SCORING_FORMAT] = ""

#VOR dataframe based on scoring format
vor_df = proj_df[['Player', 'Pos', 'Team', 'FantasyPoints']]

vor_df = vor_df.rename(columns={
    FANTASY_SCORING_FORMAT: 'Projected Fantasy Total'
})


In [ ]:
#Get dataframes of specific pos
rb_proj_df = proj_df[proj_df['Pos'] == 'RB']
wr_proj_df = proj_df[proj_df['Pos'] == 'WR']
te_proj_df = proj_df[proj_df['Pos'] == 'TE']
qb_proj_df = proj_df[(proj_df['Pos'] == 'QB') | (proj_df['Pos'] == 0)]

In [ ]:
adp_df['ADP RANK'] = adp_df['Current ADP'].rank()

adp_df_cutoff = adp_df[:100]

replacement_players = {
    'RB': '',
    'WR': '',
    'TE': '',
    'QB': ''
}


In [ ]:
for _, row in adp_df_cutoff.iterrows():
    position = row['Pos'][:2]
    player = row['Player']

    if position in replacement_players:
        replacement_players[position] = player

In [ ]:
replacement_values = {}

for position, player_name in replacement_players.items():
    player = vor_df.loc[vor_df['Player'] == player_name]
    replacement_values[position] = player['FantasyPoints'].tolist()[0]

In [ ]:
pd.set_option('chained_assignment', None)
pd.set_option('display.max_rows', None)

vor_df = vor_df.loc[vor_df['Pos'].isin(['QB', 'RB', 'WR', 'TE'])]

vor_df['VOR'] = vor_df.apply(
    lambda row: row['FantasyPoints'] - replacement_values.get(row['Pos']), axis=1
)

In [ ]:
vor_df['VOR Rank'] = vor_df['VOR'].rank(ascending=False)
# vor_df.sort_values(by='VOR Rank')

In [ ]:
min_vor = vor_df['VOR'].min()
max_vor = vor_df['VOR'].max()
vor_range = max_vor - min_vor

vor_df['VOR'] = vor_df['VOR'].apply(
    lambda x: (x - min_vor) / vor_range
)

In [ ]:
vor_df = vor_df.sort_values(by= 'VOR', ascending=False)

In [ ]:
vor_df = vor_df.rename({
    'VOR': 'Value',
    'VOR Rank': 'Value Rank',
    'Tm': 'Team'
}, axis=1)

In [ ]:
adp_df.columns

In [ ]:
# adp_df = adp_df[['Player', 'Pos', 'ADP Rank']]

# adp_df = adp_df.rename(columns={
#     'Pos': 'Pos Rank',
#     'ADP RANK': 'ADP Rank'
# })

In [ ]:
# adp_df['Pos'] = adp_df.apply(
#     lambda row: str(row['Pos Rank'])[:2]
# , axis=1)

# adp_df.head()

In [ ]:
final_df = vor_df.merge(adp_df, how='left', on=['Player', 'Pos'])

In [ ]:
final_df['Diff in ADP and Value'] = final_df['ADP Rank'] - final_df['Value Rank']
final_df = final_df.dropna()

In [ ]:
# Number of Teams * Spots on Each Team
draft_pool = final_df.sort_values(by='ADP Rank')[:196]

In [ ]:
rb_df_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'RB']
wr_df_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'WR']
te_df_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'TE']
qb_df_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'QB']

#Sleepers and Over Valued by Pos.

RBs

Sleepers

In [ ]:
rb_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=False).head(10)

Over Valued

In [ ]:
rb_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=True).head(10)

WRs

Sleepers

In [ ]:
wr_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=False).head(10)

Over Valued

In [ ]:
wr_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=True).head(10)

TEs

Sleepers

In [ ]:
te_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=False).head(10)

Over Valued

In [ ]:
te_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=True).head(10)

QBs

Sleepers

In [ ]:
qb_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=False).head(10)

Over Valued

In [ ]:
qb_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=True).head(10)

In [ ]:
final_df = final_df[['Player', 'Team', 'Value', 'Value Rank', 'ADP Rank', 'Diff in ADP and Value', 'FantasyPoints']]

In [ ]:
normalized_vor_df = final_df

normalized_vor_df['Value'] = normalized_vor_df['Value'].apply(lambda x: (x - normalized_vor_df['Value'].min()) / (normalized_vor_df['Value'].max() - normalized_vor_df['Value'].min()))

normalized_vor_df = normalized_vor_df.sort_values(by='Value Rank')
normalized_vor_df.head(100)

In [ ]:
final_df.to_csv(r'../FinalData/ValueOverReplacement_{}.csv'.format(FANTASY_SCORING_FORMAT), encoding='utf-8', index=False);